In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from scipy import stats
from sklearn import metrics 
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, RandomizedSearchCV

In [2]:
file1= "train.csv"
file2= "test.csv"
train_data= pd.read_csv(file1)
test_data= pd.read_csv(file2)
train_data.head().T

,0,1,2,3,4
Id,1,2,3,4,5
MSSubClass,60,20,60,70,60
MSZoning,RL,RL,RL,RL,RL
LotFrontage,65,80,68,60,84
LotArea,8450,9600,11250,9550,14260
...,...,...,...,...,...
MoSold,2,5,9,2,12
YrSold,2008,2007,2008,2006,2008
SaleType,WD,WD,WD,WD,WD
SaleCondition,Normal,Normal,Normal,Abnorml,Normal


In [3]:
test_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

In [5]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
Id               1459 non-null int64
MSSubClass       1459 non-null int64
MSZoning         1455 non-null object
LotFrontage      1232 non-null float64
LotArea          1459 non-null int64
Street           1459 non-null object
Alley            107 non-null object
LotShape         1459 non-null object
LandContour      1459 non-null object
Utilities        1457 non-null object
LotConfig        1459 non-null object
LandSlope        1459 non-null object
Neighborhood     1459 non-null object
Condition1       1459 non-null object
Condition2       1459 non-null object
BldgType         1459 non-null object
HouseStyle       1459 non-null object
OverallQual      1459 non-null int64
OverallCond      1459 non-null int64
YearBuilt        1459 non-null int64
YearRemodAdd     1459 non-null int64
RoofStyle        1459 non-null object
RoofMatl         1459 non-null object
Exterior1st      1458 non-

In [6]:
train_data.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [7]:
test_data.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1459.000000,1459.000000,1232.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1444.000000,1458.000000,...,1458.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000
mean,2190.000000,57.378341,68.580357,9819.161069,6.078821,5.553804,1971.357779,1983.662783,100.709141,439.203704,...,472.768861,93.174777,48.313914,24.243317,1.794380,17.064428,1.744345,58.167923,6.104181,2007.769705
std,421.321334,42.746880,22.376841,4955.517327,1.436812,1.113740,30.390071,21.130467,177.625900,455.268042,...,217.048611,127.744882,68.883364,67.227765,20.207842,56.609763,30.491646,630.806978,2.722432,1.301740
min,1461.000000,20.000000,21.000000,1470.000000,1.000000,1.000000,1879.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,1825.500000,20.000000,58.000000,7391.000000,5.000000,5.000000,1953.000000,1963.000000,0.000000,0.000000,...,318.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000
50%,2190.000000,50.000000,67.000000,9399.000000,6.000000,5.000000,1973.000000,1992.000000,0.000000,350.500000,...,480.000000,0.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,2554.500000,70.000000,80.000000,11517.500000,7.000000,6.000000,2001.000000,2004.000000,164.000000,753.500000,...,576.000000,168.000000,72.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,2919.000000,190.000000,200.000000,56600.000000,10.000000,9.000000,2010.000000,2010.000000,1290.000000,4010.000000,...,1488.000000,1424.000000,742.000000,1012.000000,360.000000,576.000000,800.000000,17000.000000,12.000000,2010.000000


In [8]:
train_data.isnull().sum()

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

In [9]:
test_data.isnull().sum()

Id                 0
MSSubClass         0
MSZoning           4
LotFrontage      227
LotArea            0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           1
SaleCondition      0
Length: 80, dtype: int64

In [10]:
missing_value= train_data.isnull().sum()
missing_value[missing_value>0]

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

In [11]:
missing_val2= test_data.isnull().sum()
missing_val2[missing_val2>0]

MSZoning           4
LotFrontage      227
Alley           1352
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType        16
MasVnrArea        15
BsmtQual          44
BsmtCond          45
BsmtExposure      44
BsmtFinType1      42
BsmtFinSF1         1
BsmtFinType2      42
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu      730
GarageType        76
GarageYrBlt       78
GarageFinish      78
GarageCars         1
GarageArea         1
GarageQual        78
GarageCond        78
PoolQC          1456
Fence           1169
MiscFeature     1408
SaleType           1
dtype: int64

In [12]:
train_data.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice          int64
Length: 81, dtype: object

In [13]:
test_data.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
                  ...   
MiscVal            int64
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
Length: 80, dtype: object

In [14]:
train_data.shape

(1460, 81)

In [15]:
test_data.shape

(1459, 80)

In [16]:
train_data.LotFrontage

0       65.0
1       80.0
2       68.0
3       60.0
4       84.0
        ... 
1455    62.0
1456    85.0
1457    66.0
1458    68.0
1459    75.0
Name: LotFrontage, Length: 1460, dtype: float64

In [17]:
train_data.GarageYrBlt

0       2003.0
1       1976.0
2       2001.0
3       1998.0
4       2000.0
         ...  
1455    1999.0
1456    1978.0
1457    1941.0
1458    1950.0
1459    1965.0
Name: GarageYrBlt, Length: 1460, dtype: float64

In [18]:
test_data.SaleType

0       WD
1       WD
2       WD
3       WD
4       WD
        ..
1454    WD
1455    WD
1456    WD
1457    WD
1458    WD
Name: SaleType, Length: 1459, dtype: object

In [19]:
#all_data= pd.concat((train_data,test_data)).reset_index(drop= False)
#all_data.drop(["SalePrice"] ,axis=1, inplace = True)
#all_data.columns

In [20]:
train_data.MasVnrArea

0       196.0
1         0.0
2       162.0
3         0.0
4       350.0
        ...  
1455      0.0
1456    119.0
1457      0.0
1458      0.0
1459      0.0
Name: MasVnrArea, Length: 1460, dtype: float64

In [21]:
test_data["MSZoning"]=test_data.MSZoning.fillna("MSZoning.mode()",inplace= True)
train_data["LotFrontage"]= train_data.LotFrontage.transform(lambda x: x.fillna(x.median()))
test_data.LotFrontage= test_data.LotFrontage.transform(lambda x: x.fillna(x.median()))
train_data.Alley=train_data.Alley.fillna("None",inplace= True)
test_data.Alley= test_data.Alley.fillna("None", inplace= True)
train_data.PoolQC= train_data.PoolQC.fillna("None",inplace= True)
test_data.PoolQC= test_data.PoolQC.fillna("None",inplace = True)
train_data.Fence= train_data.Fence.fillna("None",inplace= True)
test_data.Fence= test_data.Fence.fillna("None",inplace= True)
train_data.MiscFeature= train_data.MiscFeature.fillna("MiscFeature.mode()",inplace= True)
test_data.MiscFeature= test_data.MiscFeature.fillna("MiscFeature.mode()" , inplace= True)          
train_data.FireplaceQu= train_data.FireplaceQu.fillna("FireplaceQu.mode()",inplace= True)
test_data.FireplaceQu= test_data.FireplaceQu.fillna("FireplaceQu.mode()",inplace= True)
train_data.MasVnrType= train_data.MasVnrType.fillna("None",inplace= True)
test_data.MasVnrType= test_data.MasVnrType.fillna("None", inplace= True)
train_data.MasVnrArea= train_data.MasVnrArea.transform(lambda x: x.fillna(x.mean()))
test_data.MasVnrArea= test_data.MasVnrArea.transform (lambda x: x.fillna(x.mean()))
test_data.Exterior1st= test_data.Exterior1st.fillna("Exterior1st.mode()", inplace= True)
test_data.Exterior2nd= test_data.Exterior2nd.fillna("VinylSd", inplace= True)
test_data.Utilities= test_data.Utilities.fillna("AllPub",inplace= True)
test_data.KitchenQual= test_data.KitchenQual.fillna("KitchenQual.mode()",inplace= True)
test_data.Functional= test_data.Functional.fillna("Typ",inplace= True)
train_data.Electrical= train_data.Electrical.fillna("Electrical.mode(axis=0)",inplace= True)
test_data.SaleType= test_data.SaleType.fillna("WD",inplace= True)
test_data.GarageYrBlt= test_data.GarageYrBlt.transform(lambda x: x.fillna(x.mean()))
train_data.GarageYrBlt= train_data.GarageYrBlt.transform(lambda x: x.fillna(x.mean()))


In [22]:
for col in ["GarageType","GarageFinish","GarageQual","GarageCond"]:
    train_data[col]= train_data[col].fillna("None",inplace= True)

In [23]:
for col in ["GarageType","GarageFinish","GarageQual","GarageCond"]:
    test_data[col]= test_data[col].fillna("None",inplace= True)

In [24]:
for col in ["GarageCars","GarageArea"]:
    test_data[col]= test_data[col].transform(lambda x: x.fillna(x.mean()))

In [25]:
for col in ["BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2"]:
    train_data[col]= train_data[col].fillna("None",inplace= True)
    
for col in ["BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2"]:
    test_data[col]= test_data[col].fillna("None",inplace= True)
    
for col in ["BsmtFinSF1","BsmtFinSF2","BsmtUnfSF","BsmtFullBath","BsmtHalfBath","TotalBsmtSF"]:
    test_data[col]= test_data[col].transform(lambda x: x.fillna(x.mean()))

In [26]:
columns= ("Utilities","LotConfig","LandSlope","Neighborhood","Condition1","Condition2","HouseStyle","RoofStyle","RoofMatl","ExterQual","ExterCond","KitchenQual","SaleCondition","HeatingQC","SaleType","BldgType","Alley","MSZoning","CentralAir","Exterior1st","Exterior2nd","LandContour","LotShape","PavedDrive","Foundation","Functional","PoolQC","Fence","MiscFeature","FireplaceQu","MasVnrType","Heating","Electrical","GarageType","GarageQual","GarageFinish","GarageCond","BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2")
for x in columns:
    le= LabelEncoder()
    train_data[x]= le.fit_transform(train_data[x].values)

In [27]:
columns= ("Utilities","LotConfig","LotShape","LandSlope","Neighborhood","Condition1","Condition2","HouseStyle","RoofStyle","RoofMatl","ExterQual","ExterCond","KitchenQual","SaleCondition","HeatingQC","BldgType","Alley","MSZoning","CentralAir","Exterior1st","Exterior2nd","LandContour","PavedDrive","Foundation","Functional","PoolQC","Fence","MiscFeature","FireplaceQu","MasVnrType","Heating","Electrical","GarageType","GarageQual","GarageFinish","GarageCond","BsmtQual","BsmtCond","SaleType","BsmtExposure","BsmtFinType1","BsmtFinType2")
for y in columns:
    le= LabelEncoder()
    test_data[y]= le.fit_transform(test_data[y].values)

In [28]:
missing_val= train_data.isnull().sum()
missing_val[missing_val>0]

Series([], dtype: int64)

In [29]:
missing= test_data.isnull().sum()
missing[missing>0]

Series([], dtype: int64)

In [30]:
train_data = pd.get_dummies(train_data.drop("Id", axis = 1))
test_data=pd.get_dummies(test_data.drop("Id", axis = 1))


In [31]:
train_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,Street_Grvl,Street_Pave
0,60,3,65.0,8450,0,3,3,0,4,0,...,0,0,0,2,2008,8,4,208500,0,1
1,20,3,80.0,9600,0,3,3,0,2,0,...,0,0,0,5,2007,8,4,181500,0,1
2,60,3,68.0,11250,0,0,3,0,4,0,...,0,0,0,9,2008,8,4,223500,0,1
3,70,3,60.0,9550,0,0,3,0,0,0,...,0,0,0,2,2006,8,0,140000,0,1
4,60,3,84.0,14260,0,0,3,0,2,0,...,0,0,0,12,2008,8,4,250000,0,1


In [32]:
X= train_data.drop("SalePrice",axis= 1)
Y= train_data.SalePrice

In [33]:
train_x,val_x,train_y,val_y= train_test_split(X,Y,test_size= 0.2,random_state= 0)

In [34]:
#scaler = StandardScaler()

#train_x = scaler.fit_transform(train_x)
#val_x = scaler.fit_transform(val_x)

In [35]:
lm= LinearRegression()
val= lm.fit(train_x,train_y)
pred= val.predict(val_x)

In [36]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
print("mse:",mean_squared_error(pred,val_y))
print("r_squared:",r2_score(pred,val_y))
print("RMSE:", np.sqrt(mean_squared_error(pred,val_y)))

mse: 2889735245.4732027
r_squared: 0.6252962246150365
RMSE: 53756.25773315329


In [37]:
#from sklearn.ensemble import RandomForestRegressor
#Rf= RandomForestRegressor(n_estimators= 200,random_state= 0)
#val1= Rf.fit(train_x,train_y)
#pred1= val.predict(val_x)
#print("RMSE:",np.sqrt(mean_squared_error(pred1,val_y)))

In [38]:
#Rf.fit(X,Y)

In [39]:
#test_yhat1= Rf.predict(test_data)

In [40]:
#submission1=pd.read_csv("sample_submission.csv")
#submission1.SalePrice= test_yhat1

In [41]:
#submission1.to_csv("Presh.csv",index= False)

In [42]:
#GBR= GradientBoostingRegressor
#params= {"n_estimators" : [500, 700],
          #"learning_rate" : [0.01, 0.05, 0.5, 1],
          #"max_depth" : [5, 7],
          #"max_features" : [5, 7]}
#gsc_boost = GridSearchCV(GBR, params, cv = 5) 
#gsc_boost = gsc_boost.fit(train_x, train_y)

#print(gsc_boost.best_estimator_)
#GBR= gsc_boost.best_estimator_
#GBR.fit(train_x,train_y)
#print(round(GBR.score(val_x, val_y), 4))

In [43]:
train_data.shape

(1460, 81)

In [44]:
test_data.shape

(1459, 80)

In [45]:
lm.fit(X,Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [46]:
test_yhat= lm.predict(test_data)

In [47]:
#pearson_coef,p_value=stats.pearsonr(train_data["Id"],train_data["SalePrice"])
#print(pearson_coef,p_value)

In [48]:
#pearson_coef,p_value= stats.pearsonr(train_data["LotFrontage"],train_data["SalePrice"])
#print(pearson_coef,p_value)

In [49]:
submission= pd.read_csv("sample_submission.csv")
submission.head()

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977


In [50]:
submission.SalePrice= test_yhat
submission.head()

,Id,SalePrice
0,1461,126658.848234
1,1462,169805.105788
2,1463,182438.302993
3,1464,203022.381672
4,1465,198319.164784


In [51]:
submission.to_csv("Toyo_csv",index= False)